In [1]:
from skimage.transform import rescale
from skimage.data import shepp_logan_phantom
import numpy as np
from scipy import sparse
from besov_prior2D import besov_prior2D
from inverse_problem_2D import inverse_problem_2D
from CT import CT
from rto_mh_2D import rto_mh_2D
import pywt

In [2]:
np.random.seed(100)
J = 6
n = 2**J
N = 2**(2*J)
n_theta = 30
im_size = n
im = shepp_logan_phantom()
im_1 = rescale(im,im_size/400)
# projection angles
theta = np.linspace(0,180,n_theta,endpoint=False)
likelihood = CT(theta)
likelihood.set_data(im_1,2.0)
np.save('CT_IM.npy',im_1)
np.save("CT_data.npy",likelihood.data)
m = len(likelihood.data.ravel())

In [3]:
sparse.save_npz("A_64.npz",likelihood.jac_const(n))

In [ ]:
np.random.seed(100)
A = sparse.load_npz('A_64.npz')
s= 1.0
p = 1.5
delt = 0.025
level = 0
wavelet = 'db1'
a, slices, shape = pywt.ravel_coeffs(pywt.wavedec2(np.ones((n,n)),wavelet,mode='periodization',level=J-level))
prior = besov_prior2D(J,delt,level, slices, shape,s=s,p=p, wavelet=wavelet)
nsamp= 50000
x0 = np.ones(N)
sampler = rto_mh_2D(x0,N+m,samp=nsamp)
problem = inverse_problem_2D(likelihood, prior, A@prior.jac_const())
z_map= sampler.initialize_Q(problem)
sampler.x0 = 8*z_map
chain, acc_rate, index_accept, log_c_chain  = sampler.sample(problem)
np.save('CTSamples'+ wavelet + str(s) + str(p)+ '.npy',chain)
np.save('CTacc_rate' + wavelet + str(s) + str(p)+ '.npy',acc_rate)
np.save("CTMH" + wavelet + str(s) + str(p)+ '.npy',log_c_chain)
np.save("CTindex_accept" + wavelet + str(s) + str(p)+ '.npy', index_accept)


In [ ]:
np.random.seed(100)
A = sparse.load_npz('A_64.npz')
s= 2.5
p = 1.5
delt = 0.025
level = 0
wavelet = 'db1'
a, slices, shape = pywt.ravel_coeffs(pywt.wavedec2(np.ones((n,n)),wavelet,mode='periodization',level=J-level))
prior = besov_prior2D(J,delt,level, slices, shape,s=s,p=p, wavelet=wavelet)
nsamp= 200
x0 = np.ones(N)
sampler = rto_mh_2D(x0,N+m,samp=nsamp)
problem = inverse_problem_2D(likelihood, prior, A@prior.jac_const())
z_map= sampler.initialize_Q(problem)
sampler.x0 = 8*z_map
chain, acc_rate, index_accept, log_c_chain  = sampler.sample(problem)
np.save('CTSamples'+ wavelet + str(s) + str(p)+ '.npy',chain)
np.save('CTacc_rate' + wavelet + str(s) + str(p)+ '.npy',acc_rate)
np.save("CTMH" + wavelet + str(s) + str(p)+ '.npy',log_c_chain)
np.save("CTindex_accept" + wavelet + str(s) + str(p)+ '.npy', index_accept)


In [ ]:
np.random.seed(100)
A = sparse.load_npz('A_64.npz')
s= 2.5
p = 1.0
delt = 0.025
level = 0
wavelet = 'db1'
a, slices, shape = pywt.ravel_coeffs(pywt.wavedec2(np.ones((n,n)),wavelet,mode='periodization',level=J-level))
prior = besov_prior2D(J,delt,level, slices, shape,s=s,p=p, wavelet=wavelet)
nsamp= 200
x0 = np.ones(N)
sampler = rto_mh_2D(x0,N+m,samp=nsamp)
problem = inverse_problem_2D(likelihood, prior, A@prior.jac_const())
z_map= sampler.initialize_Q(problem)
sampler.x0 = 8*z_map
chain, acc_rate, index_accept, log_c_chain  = sampler.sample(problem)
np.save('CTSamples'+ wavelet + str(s) + str(p)+ '.npy',chain)
np.save('CTacc_rate' + wavelet + str(s) + str(p)+ '.npy',acc_rate)
np.save("CTMH" + wavelet + str(s) + str(p)+ '.npy',log_c_chain)
np.save("CTindex_accept" + wavelet + str(s) + str(p)+ '.npy', index_accept)